In [2]:
# Imports for panel visualizations

import panel as pn
import plotly.express as px
pn.extension('plotly')
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
import numpy as np

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
file_path = Path('../Data/hotel_bookings.csv')
hotel_data = pd.read_csv(file_path)
hotel_data.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [4]:
# Group by hotel and year?
grouped = hotel_data.groupby(["hotel", "arrival_date_year"])

In [5]:
avg_grouped = grouped.first()
avg_grouped.head()

is_canceled  lead_time arrival_date_month  \
hotel        arrival_date_year                                              
City Hotel   2015                         0          6               July   
             2016                         1        119            January   
             2017                         1         26            January   
Resort Hotel 2015                         0        342               July   
             2016                         0        109            January   

                                arrival_date_week_number  \
hotel        arrival_date_year                             
City Hotel   2015                                     27   
             2016                                      1   
             2017                                      1   
Resort Hotel 2015                                     27   
             2016                                      1   

                                arrival_date_day_of_month  \
hotel        arrival_date_year                              
City Hotel   2015                                       1   
             2016                                       1   
             2017                                       1   
Resort Hotel 2015                                       1   
             2016                                       1   

                                stays_in_weekend_nights  stays_in_week_nights  \
hotel        arrival_date_year                                                  
City Hotel   2015                                     0                     2   
             2016                                     0                     1   
             2017                                     1                     0   
Resort Hotel 2015                                     0                     0   
             2016                                     0                     1   

                                adults  children  babies  ... deposit_type  \
hotel        arrival_date_year                            ...                
City Hotel   2015                    1       0.0       0  ...   No Deposit   
             2016                    2       0.0       0  ...   No Deposit   
             2017                    1       0.0       0  ...   No Deposit   
Resort Hotel 2015                    2       0.0       0  ...   No Deposit   
             2016                    2       0.0       0  ...   No Deposit   

                                agent company days_in_waiting_list  \
hotel        arrival_date_year                                       
City Hotel   2015                 6.0    40.0                    0   
             2016                14.0    14.0                    0   
             2017                 9.0    40.0                    0   
Resort Hotel 2015               304.0   110.0                    0   
             2016               240.0   174.0                    0   

                                  customer_type     adr  \
hotel        arrival_date_year                            
City Hotel   2015                     Transient    0.00   
             2016                     Transient   74.25   
             2017                     Transient  128.00   
Resort Hotel 2015                     Transient    0.00   
             2016               Transient-Party   59.94   

                                required_car_parking_spaces  \
hotel        arrival_date_year                                
City Hotel   2015                                         0   
             2016                                         0   
             2017                                         0   
Resort Hotel 2015                                         0   
             2016                                         0   

                               total_of_special_requests reservation_status  \
hotel        arrival_date_year                                                
City Hotel   2015      

In [6]:
avg_grouped.hvplot.bar(x="arrival_date_year", y="lead_time", groupby="hotel")

:DynamicMap   [hotel]
   :Bars   [arrival_date_year]   (lead_time)

In [7]:
# Map country of origin?

In [8]:
hotel_data["country"].value_counts().sort_values(ascending=False)

PRT    48590
GBR    12129
FRA    10415
ESP     8568
DEU     7287
ITA     3766
IRL     3375
BEL     2342
BRA     2224
NLD     2104
USA     2097
CHE     1730
CN      1279
AUT     1263
SWE     1024
CHN      999
POL      919
ISR      669
RUS      632
NOR      607
ROU      500
FIN      447
DNK      435
AUS      426
AGO      362
LUX      287
MAR      259
TUR      248
HUN      230
ARG      214
       ...  
HND        1
MDG        1
CYM        1
ASM        1
LCA        1
SMR        1
DJI        1
BFA        1
BWA        1
VGB        1
FJI        1
NPL        1
NIC        1
BDI        1
NAM        1
UMI        1
NCL        1
BHS        1
ATF        1
SDN        1
MLI        1
PLW        1
KIR        1
MMR        1
DMA        1
SLE        1
AIA        1
GUY        1
PYF        1
MRT        1
Name: country, Length: 177, dtype: int64

In [9]:
load_dotenv()
mapbox_token = os.getenv("MAPBOX")

In [10]:
px.set_mapbox_access_token(mapbox_token)

In [11]:
file_path = Path("worldcities.csv")
world_data = pd.read_csv(file_path)
world_data.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140


In [12]:
world_data = world_data.drop(columns=["city_ascii", "iso2", "iso3", "iso3", "admin_name", "capital", "id", "population"])

In [13]:
world_data.head()

,city,lat,lng,country
0,Tokyo,35.6897,139.6922,Japan
1,Jakarta,-6.2146,106.8451,Indonesia
2,Delhi,28.6600,77.2300,India
3,Mumbai,18.9667,72.8333,India
4,Manila,14.5958,120.9772,Philippines


In [14]:
world_data.hvplot.points("lng", "lat", geo=True, color="yellow", alpha=0.2, tiles="ESRI")

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [lng,lat]

In [15]:
# Replace with hotel data and have functions that return figures, plots, etc.

# avg_data = pn.Row(average_gross_rent(), average_sales_price())
# yearly_data = pn.Column("replace with relevant hotel information")
# column_of_ML_model_1 = pn.Column(replace with ML model 1 information)
# neighborhood_data = pn.Column(column_of_neighborhood) 
# world_map_data = pn.Column(world_map()) 


# panel = pn.Tabs(
#    ("Yearly Market", yearly_data),   
#    ("Country of Origin Map", world_map_data),
#    ("ML model 1 Analysis", ML_model_1_data),
# )

In [ ]:
# Serve the Panel dashboard

# panel.servable()